# Capabilities in OpenDXA

This tutorial covers the capabilities system in OpenDXA, which provides a way to define and manage agent capabilities.

## Learning Objectives

By the end of this tutorial, you will understand:

1. How to define agent capabilities
2. How to create custom capabilities
3. How to compose capabilities
4. How to manage capability discovery and registration
5. How to version and update capabilities
6. How to test and validate capabilities

## Prerequisites

- Basic understanding of OpenDXA's architecture
- Familiarity with Python classes and inheritance
- Understanding of basic resource management concepts

## 1. Understanding Capabilities

Capabilities in OpenDXA are modular components that define what an agent can do. They can be:

1. **Basic Capabilities**: Simple, single-function capabilities
2. **Composite Capabilities**: Combinations of multiple capabilities
3. **Domain-Specific Capabilities**: Specialized for particular domains

Let's start by creating some basic capabilities:

In [ ]:
from opendxa.agent.capability import BaseCapability
from typing import Dict, Any, Optional

class DataAnalysisCapability(BaseCapability):
    """Example capability for data analysis."""
    def __init__(self, name: str = "data_analysis"):
        super().__init__(name)
        self.description = "Analyzes data using various statistical methods"
        self.version = "1.0.0"
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute data analysis."""
        # Example implementation
        return {
            "mean": sum(data.get("values", [])) / len(data.get("values", [])),
            "count": len(data.get("values", []))
        }

class VisualizationCapability(BaseCapability):
    """Example capability for data visualization."""
    def __init__(self, name: str = "visualization"):
        super().__init__(name)
        self.description = "Creates visualizations from data"
        self.version = "1.0.0"
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Create visualization."""
        # Example implementation
        return {
            "plot_type": data.get("plot_type", "line"),
            "data_points": len(data.get("values", []))
        }

## 2. Creating Custom Capabilities

Let's create more sophisticated custom capabilities with additional features:

In [ ]:
class MachineLearningCapability(BaseCapability):
    """Example capability for machine learning tasks."""
    def __init__(self, name: str = "machine_learning"):
        super().__init__(name)
        self.description = "Performs machine learning tasks"
        self.version = "1.0.0"
        self.supported_models = ["classification", "regression", "clustering"]
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute machine learning task."""
        model_type = data.get("model_type")
        if model_type not in self.supported_models:
            raise ValueError(f"Unsupported model type: {model_type}")
            
        # Example implementation
        return {
            "model_type": model_type,
            "training_samples": len(data.get("training_data", [])),
            "features": len(data.get("features", []))
        }
        
    def validate_input(self, data: Dict[str, Any]) -> bool:
        """Validate input data."""
        required_fields = ["model_type", "training_data", "features"]
        return all(field in data for field in required_fields)

# Create and use the capability
ml_capability = MachineLearningCapability()

# Example data
data = {
    "model_type": "classification",
    "training_data": [1, 2, 3, 4, 5],
    "features": ["feature1", "feature2"]
}

# Validate and execute
if ml_capability.validate_input(data):
    result = await ml_capability.execute(data)
    print(f"ML task result: {result}")
else:
    print("Invalid input data")

## 3. Capability Composition

Capabilities can be composed to create more complex functionality. Let's see how to do this:

In [ ]:
class CompositeCapability(BaseCapability):
    """Example of a composite capability."""
    def __init__(self, name: str, capabilities: Dict[str, BaseCapability]):
        super().__init__(name)
        self.description = "Combines multiple capabilities"
        self.version = "1.0.0"
        self.capabilities = capabilities
        
    async def execute(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute composite capability."""
        results = {}
        for name, capability in self.capabilities.items():
            if name in data:
                results[name] = await capability.execute(data[name])
        return results

# Create individual capabilities
analysis = DataAnalysisCapability()
visualization = VisualizationCapability()
ml = MachineLearningCapability()

# Create composite capability
data_science = CompositeCapability(
    "data_science",
    {
        "analysis": analysis,
        "visualization": visualization,
        "machine_learning": ml
    }
)

# Example data for each capability
data = {
    "analysis": {"values": [1, 2, 3, 4, 5]},
    "visualization": {"plot_type": "line", "values": [1, 2, 3, 4, 5]},
    "machine_learning": {
        "model_type": "classification",
        "training_data": [1, 2, 3, 4, 5],
        "features": ["feature1", "feature2"]
    }
}

# Execute composite capability
results = await data_science.execute(data)
print("Composite capability results:")
for name, result in results.items():
    print(f"{name}: {result}")

## 4. Capability Discovery and Registration

OpenDXA provides a system for discovering and registering capabilities. Let's implement this:

In [ ]:
class CapabilityRegistry:
    """Example capability registry."""
    def __init__(self):
        self.capabilities: Dict[str, BaseCapability] = {}
        
    def register(self, capability: BaseCapability) -> None:
        """Register a capability."""
        self.capabilities[capability.name] = capability
        
    def get_capability(self, name: str) -> Optional[BaseCapability]:
        """Get a capability by name."""
        return self.capabilities.get(name)
        
    def list_capabilities(self) -> Dict[str, Dict[str, Any]]:
        """List all registered capabilities."""
        return {
            name: {
                "description": cap.description,
                "version": cap.version
            }
            for name, cap in self.capabilities.items()
        }

# Create and use the registry
registry = CapabilityRegistry()

# Register capabilities
registry.register(DataAnalysisCapability())
registry.register(VisualizationCapability())
registry.register(MachineLearningCapability())

# List registered capabilities
print("Registered capabilities:")
for name, info in registry.list_capabilities().items():
    print(f"{name}: {info['description']} (v{info['version']})")

## 5. Capability Versioning and Updates

Let's implement versioning and update management for capabilities:

In [ ]:
from typing import List, Tuple
import semver

class VersionedCapability(BaseCapability):
    """Example of a versioned capability."""
    def __init__(self, name: str, version: str):
        super().__init__(name)
        self.version = version
        self.description = f"Versioned capability {version}"
        self.changelog: List[Tuple[str, str]] = []  # (version, description)
        
    def add_change(self, version: str, description: str) -> None:
        """Add a changelog entry."""
        if semver.compare(version, self.version) > 0:
            self.changelog.append((version, description))
            
    def get_changes_since(self, version: str) -> List[Tuple[str, str]]:
        """Get changes since a specific version."""
        return [
            (v, desc) for v, desc in self.changelog
            if semver.compare(v, version) > 0
        ]

# Create a versioned capability
capability = VersionedCapability("example", "1.0.0")

# Add some changes
capability.add_change("1.1.0", "Added new feature X")
capability.add_change("1.2.0", "Fixed bug Y")
capability.add_change("2.0.0", "Major API changes")

# Get changes since version 1.0.0
changes = capability.get_changes_since("1.0.0")
print("Changes since version 1.0.0:")
for version, description in changes:
    print(f"{version}: {description}")

## 6. Testing and Validation

Let's implement testing and validation for capabilities:

In [ ]:
class CapabilityTester:
    """Example capability tester."""
    def __init__(self):
        self.tests: Dict[str, List[Dict[str, Any]]] = {}
        
    def add_test(self, capability_name: str, test_data: Dict[str, Any]) -> None:
        """Add a test case."""
        if capability_name not in self.tests:
            self.tests[capability_name] = []
        self.tests[capability_name].append(test_data)
        
    async def run_tests(self, registry: CapabilityRegistry) -> Dict[str, List[Dict[str, Any]]]:
        """Run all tests."""
        results = {}
        for name, tests in self.tests.items():
            capability = registry.get_capability(name)
            if not capability:
                continue
                
            results[name] = []
            for test in tests:
                try:
                    result = await capability.execute(test["input"])
                    results[name].append({
                        "status": "success",
                        "result": result
                    })
                except Exception as e:
                    results[name].append({
                        "status": "error",
                        "error": str(e)
                    })
                    
        return results

# Create test cases
tester = CapabilityTester()

# Add test for data analysis
tester.add_test("data_analysis", {
    "input": {"values": [1, 2, 3, 4, 5]},
    "expected": {"mean": 3.0, "count": 5}
})

# Add test for visualization
tester.add_test("visualization", {
    "input": {"plot_type": "line", "values": [1, 2, 3]},
    "expected": {"plot_type": "line", "data_points": 3}
})

# Run tests
results = await tester.run_tests(registry)
print("Test results:")
for name, tests in results.items():
    print(f"\n{name}:")
    for i, test in enumerate(tests):
        print(f"  Test {i + 1}: {test['status']}")
        if test['status'] == 'success':
            print(f"    Result: {test['result']}")
        else:
            print(f"    Error: {test['error']}")

## 7. Best Practices

Here are some best practices for working with capabilities in OpenDXA:

1. **Capability Design**
   - Keep capabilities focused and single-purpose
   - Use clear and descriptive names
   - Document capabilities thoroughly

2. **Version Management**
   - Use semantic versioning
   - Maintain a changelog
   - Document breaking changes

3. **Testing**
   - Write comprehensive tests
   - Test edge cases
   - Validate inputs and outputs

4. **Composition**
   - Compose capabilities logically
   - Handle errors appropriately
   - Maintain clear interfaces

5. **Discovery and Registration**
   - Use a central registry
   - Implement proper error handling
   - Support capability lookup

## Summary

In this tutorial, we covered:

1. How to define and create capabilities
2. How to compose capabilities
3. How to manage capability discovery and registration
4. How to version and update capabilities
5. How to test and validate capabilities
6. Best practices for working with capabilities

The capabilities system in OpenDXA provides a flexible and extensible way to define and manage agent functionality. By following the best practices outlined in this tutorial, you can create robust and maintainable capabilities for your OpenDXA applications.